In [1]:
from fastai.vision.all import *
from fastai.vision.widgets import *

In [ ]:
#This is the function that runs 
out_pl = widgets.Output()
lbl_pred = widgets.Label()
btn_run = widgets.Button(description='Classify')
btn_upload = widgets.FileUpload()

In [ ]:
learn_inf = load_learner('export.pkl')

In [ ]:
def on_click_classify():
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify) #Pass on_click_classify function to the on_click of our button

In [3]:
#hide_output
#VBox puts it all together in an array of buttons, labels, etc
VBox([widgets.Label('Select your bear!'), 
      btn_upload, btn_run, out_pl, lbl_pred])

In [4]:
#https://mybinder.org/v2/gh/finnisvanilla/bear_viola_test/master?urlpath=%2Fvoila%2Frender%2FBears_Viola.ipynb